### Description

In this task you will work with fault injection before the last Sbox operation. 

This fault model is used in practice. A fault shall be injected during algorithm run, so, this fault can be considered as dynamic.

Any fault injection method, i.e, a glitch, laser, EM, etc., can achieve this kind of effect.


### Fault model

This is a fault with an unknown distribution.

s' = s ^ error

where s is a byte of the AES128 State just before the last Sbox
error is unknown

### Task:

Your task is to find a Master key (round key 0) embedded into ../aes128 binary. 

The master key is in the form of HEIG{XXXXXXXXX}, where X is a ASCII printable symbol.

### Fault illustration

<img src="support/fault.png">

In [ ]:
import numpy as np
import binascii
import string
import random
import socket

HOST = 'iict-mv330-sfa'
PORT = 5003

import tqdm
import sca_training

def binary_aes128_encrypt_fault(plaintext, fault_ind, verbose=False):
    import subprocess
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)
    
    c_ctext = binascii.unhexlify(output[10:42])
    f_ctext = binascii.unhexlify(output[53:85])
    xor =  int.from_bytes(c_ctext, byteorder = 'big') ^ int.from_bytes(f_ctext, byteorder = 'big')
    
    if (verbose):
        print('Full output:', output.strip())
        print('Correct ciphertext: {}'.format(c_ctext.hex()) )
        print('Faulted ciphertext: {}'.format(f_ctext.hex()) )
        print('Difference (XOR):   {:032X}'.format(xor) )
    
    return output, c_ctext, f_ctext

In [ ]:
ptext = "000102030405060708090a0b0c0d0e0F"
output, c_ctext, f_ctext = binary_aes128_encrypt_fault(ptext, 15, verbose=True)

In [ ]:
#Get master key using the provided binary
key_schedule = sca_training.inverse_key_expansion(round_key)
print(key_schedule.shape)

print('Master key in hex:', key_schedule[0,0,:])

print('Master key in ASCII:', binascii.unhexlify(''.join('{:02x}'.format(c) for c in key_schedule[0,0,:])))